In [ ]:
! pip install pytorch-lightning==1.18
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev
!pip install -q nnAudio
!pip install -q ttach
!pip install fsspec
!pip install gcsfs
!pip install albumentations --upgrade
!pip install timm

In [ ]:
# ====================================================
# Library
# ====================================================

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

import pytorch_lightning as pl
from torchmetrics.functional import auroc

import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations import ImageOnlyTransform

import timm

from torch.cuda.amp import autocast, GradScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

train = pd.read_csv('/content/training_labels.csv')
test = pd.read_csv('/content/sample_submission.csv')

def get_train_file_path(image_id):
    return "/content/train/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

def get_test_file_path(image_id):
    return "/content/test/{}/{}/{}/{}.npy".format(
        image_id[0], image_id[1], image_id[2], image_id)

train['file_path'] = train['id'].apply(get_train_file_path)
test['file_path'] = test['id'].apply(get_test_file_path)

display(train.head())
display(test.head())

In [ ]:
def get_folds(df,folds):
    x = df.id.values
    y = df["target"].values    
    mskf = StratifiedKFold(n_splits = folds)    
    for fold, (trn_, val_) in enumerate(mskf.split(x, y)):
        print("TRAIN: ", trn_, "VAL: ",  val_)
        df.loc[val_, "kfold"] = fold
    return df

In [ ]:
train=get_folds(train,5)

In [ ]:
train.head()

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    apex=True
    debug=False
    num_workers=40
    learning_rate=0.001
    model_name='efficientnet_b3a'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=6
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    img_size=340
    min_lr=1e-6
    batch_size=16
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"sr": 2048, "fmin": 20, "fmax": 1024, "hop_length": 32, "bins_per_octave": 8, "verbose": False}
    seed=2021
    target_size=2
    target_col='target'
    n_fold=5
    fold=0 # [0, 1, 2, 3, 4]
    train=True
    
if CFG.debug:
    CFG.epochs = 1
    train = train.sample(n=10000, random_state=CFG.seed).reset_index(drop=True)

In [ ]:
import torch
from nnAudio.Spectrogram import CQT1992v2

def apply_qtransform(waves, transform=CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)):
    waves = np.hstack(waves)
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()
    image = transform(waves)
    return image

for i in range(5):
    waves = np.load(train.loc[i, 'file_path'])
    image = apply_qtransform(waves)
    print(image.shape)
    target = train.loc[i, 'target']
    plt.imshow(image[0])
    plt.title(f"target: {target}")
    plt.show()

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

In [ ]:
# ====================================================
# Transforms
# ====================================================
def get_transforms(*, data):
    
    if data == 'train':
        return A.Compose([
            A.augmentations.geometric.resize.Resize(CFG.img_size,CFG.img_size),
            ToTensorV2()
        ])

    elif data == 'valid':
        return A.Compose([
            A.augmentations.geometric.resize.Resize(CFG.img_size,CFG.img_size),
            ToTensorV2()
        ])

In [ ]:
# ====================================================
# Dataset
# ====================================================
class g2netDataset(Dataset):
    def __init__(self, df, mode, transform=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.mode=mode
        if self.mode=="train":
            self.labels = df[CFG.target_col].values
        self.wave_transform = CQT1992v2(**CFG.qtransform_params)
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def apply_qtransform(self, waves, transform):
        waves = np.hstack(waves)
        waves = waves / np.max(waves)
        waves = torch.from_numpy(waves).float()
        image = transform(waves)
        return image

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        waves = np.load(file_path)
        image = self.apply_qtransform(waves, self.wave_transform)
        image = image.squeeze().numpy()
        if self.transform:
            image = self.transform(image=image)['image']
        if self.mode=="train":
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return image, label
        else:
            return image

In [ ]:
class g2netDataModule(pl.LightningDataModule):
    def __init__(self, df,test_df,cfg=CFG):
        super().__init__()
        self.df=df
        self.test_df=test_df
        self.cfg=cfg
        self.fold=self.cfg.fold

    def setup(self, stage= None):
        if stage=="fit" or stage is None:
            train_df=self.df.loc[self.df.kfold!=self.fold]
            val_df=self.df.loc[self.df.kfold==self.fold]
            train_df=train_df.reset_index(drop=True)
            val_df=val_df.reset_index(drop=True)
            self.train_dataset=g2netDataset(train_df,transform=get_transforms(data="train"),mode="train")
            self.val_dataset=g2netDataset(val_df,transform=get_transforms(data="valid"),mode="train")
            
        if stage=="predict" or stage is None:
            self.test_dataset=g2netDataset(self.test_df,transform=get_transforms(data="valid"),mode="test")
        
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset,
                                           batch_size=self.cfg.batch_size,
                                           num_workers=self.cfg.num_workers,
                                           shuffle=True)
        
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset,
                                           batch_size=self.cfg.batch_size,
                                           num_workers=self.cfg.num_workers,
                                           shuffle=False) 
    
    def prdict_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset,
                                           batch_size=self.cfg.batch_size,
                                           num_workers=self.cfg.num_workers,
                                           shuffle=False) 

In [ ]:
def epoch_update_gamma(y_true,y_pred, epoch=-1,delta=1):
        """
        Calculate gamma from last epoch's targets and predictions.
        Gamma is updated at the end of each epoch.
        y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
        y_pred: `Tensor` . Predictions.
        """
        DELTA = delta+1
        SUB_SAMPLE_SIZE = 2000.0
        pos = y_pred[y_true==1]
        neg = y_pred[y_true==0] # yo pytorch, no boolean tensors or operators?  Wassap?
        # subsample the training set for performance
        cap_pos = pos.shape[0]
        cap_neg = neg.shape[0]
        pos = pos[torch.rand_like(pos) < SUB_SAMPLE_SIZE/cap_pos]
        neg = neg[torch.rand_like(neg) < SUB_SAMPLE_SIZE/cap_neg]
        ln_pos = pos.shape[0]
        ln_neg = neg.shape[0]
        pos_expand = pos.view(-1,1).expand(-1,ln_neg).reshape(-1)
        neg_expand = neg.repeat(ln_pos)
        diff = neg_expand - pos_expand
        ln_All = diff.shape[0]
        Lp = diff[diff>0] # because we're taking positive diffs, we got pos and neg flipped.
        ln_Lp = Lp.shape[0]-1
        diff_neg = -1.0 * diff[diff<0]
        diff_neg = diff_neg.sort()[0]
        ln_neg = diff_neg.shape[0]-1
        ln_neg = max([ln_neg, 0])
        left_wing = int(ln_Lp*DELTA)
        left_wing = max([0,left_wing])
        left_wing = min([ln_neg,left_wing])
        default_gamma=torch.tensor(0.2, dtype=torch.float).cuda()
        if diff_neg.shape[0] > 0 :
           gamma = diff_neg[left_wing]
        else:
           gamma = default_gamma # default=torch.tensor(0.2, dtype=torch.float).cuda() #zoink
        L1 = diff[diff>-1.0*gamma]
        ln_L1 = L1.shape[0]
        if epoch > -1 :
            return gamma
        else :
            return default_gamma



def roc_star_loss( _y_true, y_pred, gamma, _epoch_true, epoch_pred):
        """
        Nearly direct loss function for AUC.
        See article,
        C. Reiss, "Roc-star : An objective function for ROC-AUC that actually works."
        https://github.com/iridiumblue/articles/blob/master/roc_star.md
            _y_true: `Tensor`. Targets (labels).  Float either 0.0 or 1.0 .
            y_pred: `Tensor` . Predictions.
            gamma  : `Float` Gamma, as derived from last epoch.
            _epoch_true: `Tensor`.  Targets (labels) from last epoch.
            epoch_pred : `Tensor`.  Predicions from last epoch.
        """
        #convert labels to boolean
        y_true = (_y_true>=0.50)
        epoch_true = (_epoch_true>=0.50)

        # if batch is either all true or false return small random stub value.
        if torch.sum(y_true)==0 or torch.sum(y_true) == y_true.shape[0]: return torch.sum(y_pred)*1e-8

        pos = y_pred[y_true]
        neg = y_pred[~y_true]

        epoch_pos = epoch_pred[epoch_true]
        epoch_neg = epoch_pred[~epoch_true]

        # Take random subsamples of the training set, both positive and negative.
        max_pos = 1000 # Max number of positive training samples
        max_neg = 1000 # Max number of positive training samples
        cap_pos = epoch_pos.shape[0]
        cap_neg = epoch_neg.shape[0]
        epoch_pos = epoch_pos[torch.rand_like(epoch_pos) < max_pos/cap_pos]
        epoch_neg = epoch_neg[torch.rand_like(epoch_neg) < max_neg/cap_pos]

        ln_pos = pos.shape[0]
        ln_neg = neg.shape[0]

        # sum positive batch elements agaionst (subsampled) negative elements
        if ln_pos>0 :
            pos_expand = pos.view(-1,1).expand(-1,epoch_neg.shape[0]).reshape(-1)
            neg_expand = epoch_neg.repeat(ln_pos)

            diff2 = neg_expand - pos_expand + gamma
            l2 = diff2[diff2>0]
            m2 = l2 * l2
            len2 = l2.shape[0]
        else:
            m2 = torch.tensor([0], dtype=torch.float).cuda()
            len2 = 0

        # Similarly, compare negative batch elements against (subsampled) positive elements
        if ln_neg>0 :
            pos_expand = epoch_pos.view(-1,1).expand(-1, ln_neg).reshape(-1)
            neg_expand = neg.repeat(epoch_pos.shape[0])

            diff3 = neg_expand - pos_expand + gamma
            l3 = diff3[diff3>0]
            m3 = l3*l3
            len3 = l3.shape[0]
        else:
            m3 = torch.tensor([0], dtype=torch.float).cuda()
            len3=0

        if (torch.sum(m2)+torch.sum(m3))!=0 :
           res2 = torch.sum(m2)/max_pos+torch.sum(m3)/max_neg
           #code.interact(local=dict(globals(), **locals()))
        else:
           res2 = torch.sum(m2)+torch.sum(m3)

        res2 = torch.where(torch.isnan(res2), torch.zeros_like(res2), res2)

        return res2
    
class ROC_Star(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, y_pred,_y_true,i):    #_epoch_true, epoch_pred):
        return roc_star_loss( _y_true, y_pred, CFG.gamma, torch.from_numpy(CFG.last_epoch_true[i]).cuda(), torch.from_numpy(CFG.last_epoch_pred[i]).cuda())

In [ ]:
loss_fn=nn.CrossEntropyLoss()

In [ ]:
class g2netModel(pl.LightningModule):
    def __init__(self,cfg=CFG, pretrained=True):
        super(g2netModel,self).__init__()
        self.cfg = cfg
        self.lr=self.cfg.learning_rate
        self.model = timm.create_model(self.cfg.model_name, pretrained=pretrained,in_chans=1)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(self.n_features, self.cfg.target_size)

    def forward(self, x):
        output = self.model(x)
        return output

    def training_step(self,batch,batch_nb):
        x, y= batch
        logits=nn.Sigmoid()(self(x))
        loss=loss_fn(logits,y)
        acc=auroc(logits,y,num_classes=self.cfg.target_size)
        self.log("train auroc", acc, prog_bar=True)
        return loss
    
    def validation_step(self,batch,batch_nb):
        x, y= batch
        logits=nn.Sigmoid()(self(x))
        loss=loss_fn(logits,y)
        acc=auroc(logits,y,num_classes=self.cfg.target_size)
        self.log("val loss", loss, prog_bar=True)
        self.log("val auroc", acc, prog_bar=True)
        return loss
    
    def predict_step(self,batch,batch_nb):
        x= batch
        logits=nn.Sigmoid()(self(x))
        x=x[:,1].view(x.size(0),-1)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min")
        return {
            "optimizer":optimizer,
            "lr_scheduler":{
                "scheduler":scheduler,
                "monitor":"val loss"
            }
        }

In [ ]:
dm=g2netDataModule(train,test,CFG)

In [ ]:
g2netModel=g2netModel(CFG)

trainer = pl.Trainer(max_epochs = CFG.epochs,
                    gpus=1,
                    progress_bar_refresh_rate=1,
                   precision=16,
                   default_root_dir="./")
trainer.fit(g2netModel,dm)    